# M5.4 - Writing a Transparent Algorithm

*Part of:* [**Open Climate Science for Crops & Crop Conditions**](https://github.com/OpenClimateScience/M5-Open-Science-for-Crops)

In [ ]:
import datetime
import glob
import numpy as np
import xarray as xr
import rasterio
import rioxarray
from matplotlib import pyplot
from rasterio.warp import Resampling

DATE_START = datetime.date(2023, 10, 1)
DATE_SOWING = datetime.date(2023, 10, 15)
DATE_END = datetime.date(2024, 9, 29)

## Overview

We're now ready to compute the water requirement satisfaction index (WRSI). Because WRSI is a moderately complex algorithm, it's important that we write computer code that satisfies the following criteria:

- The code should correctly implement the WRSI algorithm, of course.
- Anyone reading the code should be able to understand what each line of code does.
- Each part of the code should be clearly related to a specific part of the algorithm's written description, as applicable.
- Appropriate physical units for inputs, outputs, and important intermediate variables should be made clear.

In this lesson, we'll see examples of code that satisfies these criteria, making it easy for humans to read and to verify that the code is working as expected.

### The meaning of "correct"

It's essential that our computer code *correctly* implement the algorithm or model we're working with. By "correctly," we mean that the computer code does what is intended. We do *not* mean that the results it produces are accurate when compared with some objective state or real-world measurement. 

While accuracy is important, it is secondary to correct implementation. After all, if our computer model is both incorrect and inaccurate, it will not be possible to improve its accuracy until we have a correct implementation. Imagine you are try to figure out if there is a problem with your car's tail lights. As you are standing behind the car, you ask your friend to switch on the right-turn signal, but your friend turns the left-turn signal on instead. If your friend isn't turning on the correct signal, it's going to be difficult to figure out what problem you might have with the tail lights. Therefore, a correct implementation is necessary to diagnose what needs to be improved.

**The idea of a correct implementation assumes there is some *authoritative description* of the algorithm, usually a written document in a natural language.** While a software implementation of an algorithm can be authoritative, we usually start with a written description of an algorithm in a white paper or peer-review publication.

## Compiling our PET data

In the previous lesson, we projected our evapotranspiration (ET) and potential ET (PET) data onto a 1-km global EASE-Grid 2.0 and compiled it into a single `xarray` dataset. Let's open that file, along with our land-cover classification.

In [ ]:
with rasterio.open('data/processed/MODIS_MCD12Q1_Type5_cereal_croplands_h18v05_2023.tiff') as dataset:
    lc_map = dataset.read(1)

ds_et = xr.open_dataset('data/processed/VNP16_ET_and_PET_mm_day-1.nc4')

Next, because our ET and PET data are based on 8-day composites, let's interpolate the data to daily time steps using the `resample()` method of an `xarray` dataset. We call `ffill()` after `resample()` to specify that the missing values (between each original 8-day step) should be filled in by copying the previous value forward ("forward-filling").

In [ ]:
ds_et_daily = ds_et.where(lc_map == 1, np.nan).resample(time = '1D').ffill()
ds_et_daily

---

## Computing the initial soil water content

The last input we need in order to compute the WRSI is the initial soil water content, i.e., at time step $t=0$. The WRSI is a water balance calculation, [similar to what we did in Module 3,](https://github.com/OpenClimateScience/M3-Open-Science-for-Water-Resources) so we need to know how much water is initially in "soil water" bucket. We will approximate this by assuming that precipitation and ET balance each other on an annual time scale; that is, neglecting runoff, we assume:
$$
\sum^{\text{1 year}} \text{Soil water} = \sum^{\text{1 year}} \text{Precipitation} - \sum^{\text{1 year}} \text{ET}
$$

This is a crude assumption, but it turns out the WRSI isn't especially sensitive to the initial SWC.

In order to calculate the initial soil water content, we need to load our precipitation data. In the previous lesson, we projected the IMERG-Final precipitation data onto a 9-km global EASE-Grid 2.0, for convenience and to reduce storage and computing requirements. Now, we need to resample the data again to match the 1-km resolution of our ET and PET data.

In [ ]:
# NOTE: This might take several seconds
example = rioxarray.open_rasterio('data/processed/VNP16_ET_and_PET/VNP16_ET_mm_8day-1_20230930.tiff')

# Resample the precip data to match our ET data
ds_precip = xr.open_dataset('data/processed/IMERG_precip_mm_day-1_for_study_area.nc4')\
    .rio.write_crs(example.rio.crs)
ds_precip_1km = ds_precip.rio.reproject_match(example, resampling = Resampling.bilinear)

Note that we have a 366-day record of precipitation.

In [ ]:
ds_precip_1km.time.size

And note that this matches our daily ET and PET data.

In [ ]:
ds_et_daily.time.size

Therefore, we can compute the total annual precipitation (over a 366-day annual cycle) by taking the sum over the `'time'` dimension:

In [ ]:
precip_total = ds_precip_1km.sum('time').precipitation

We do the same with ET and subtract it from the mean annual precipitation. The result is the expected amount of water (mm) that could be stored in the soil. To get the right daily magnitude, we'll divide this annual residual by the number of days in the year (366).

In [ ]:
# Our best guess for the soil water content might be
#    the mean annual balance between precip and ET

swc = (precip_total - ds_et_daily.ET.sum('time')) / ds_precip_1km.time.size # (366 days)
swc

In [ ]:
swc.where(swc >= 0, 0).where(lc_map == 1, np.nan).plot()

---

## Field capacity, rooting depth, and crop coefficients

In many scientific models, there are a number of coefficients that must be specified based on field measurements, the literature, or theory. In the WRSI, there are multiple coefficients that represent the influence of soil characteristics on the soil water balance.

The critical soil water level (SWC) is given by:
$$
\text{SWC}(t) = \text{FC}\times \text{SW}_f\times \text{RD}_f(t)
$$

Field capacity (FC) depends on soil texture. While we could use a map of soil textures, for simplicity, we'll assume the soils are a sandy loam, with a median field capacity (from [FAO 56 Table 19](https://www.fao.org/4/X0490E/x0490e0c.htm#soil%20evaporation%20reduction%20coefficient%20(kr))) of 0.23 m$^3$ m$^{-3}$. If the maximum rooting depth of wheat is about 1.5 m ([Fan et al. 2016, *Field Crops Research*](https://doi.org/10.1016/j.fcr.2016.02.013)), then this corresponds to:
$$
\text{FC} = \frac{0.23\, \text{m}^3\text{m}^{-3}}{1.5\,\text{m}\times 1.0\,\text{m} \times 1.0\,\text{m}} = 0.153 \,\text{m} = 153 \,\text{mm}
$$

Again, referring to the FAO 56 document for representative soil characteristics, according to [FAO 56 (Table 22)](https://www.fao.org/4/X0490E/x0490e0e.htm#readily%20available%20water%20(raw)), for Spring Wheat, $\text{SW}_f = 1 - 0.55 = 0.45$.


$\text{RD}_f$ varies over time as the crop's roots develop. For wheat in Northern Algeria, sowing generally occurs in October or November, with the crop reaching maturity between February and April. Without knowing the conditions in individual fields, we'll assume sowing occured October 15 and the crop reached full maturity on April 1. Therefore, we'll have $\text{RD}_f$ increase from 0.1 on October 15 to 1.0 on April 1.

For the crop coefficient, we'll use values from [FAO 56 Table 12.](https://www.fao.org/4/X0490E/x0490e0b.htm#tabulated%20kc%20values) There are three coefficients, depending on crop stage (initial, middle, and end). For Spring Wheat, these values are 0.3, 1.15, and 0.25; we'll assume that these crop stages occur on October 15, April 1, and June 1, respectively.

&#x1F449; **As you can see, there's a lot of background information that goes into a complex algorithm like the WRSI.** We'll soon see how this information should be properly attributed in our code.

### Computing the crop coefficient

We'll need to turn our crop coefficients into a daily time series, for compatibility with our algorithm that uses a daily time step. The `linear_interp()` function, below, creates a complete, daily time series based on a finite number of days with known values.

In [ ]:
def linear_interp(doy, known_values, known_doy):
    '''
    Linearly interpolates between coefficients on known dates.

    Parameters
    ----------
    doy : Sequence
        Integer sequence of day-of-year
    known_values : Sequence
        The known values (on specific days)
    known_doy : Sequence
        The day-of-year corresponding to each value in `known_values`

    Returns
    -------
    Sequence
    '''
    # Place the crop coefficients into the empty DOY time series
    arr = np.nan * np.ones(doy.shape)
    arr[np.in1d(doy, known_doy)] = known_values
    
    # Get the ordinal positions of each DOY value for which we know
    #    the crop coefficient
    x_loc = np.argwhere(np.in1d(doy, known_doy)).ravel()
    # Interpolate between the known crop coefficients
    return np.interp(np.arange(0, doy.size), x_loc, known_values)

In [ ]:
# We linearly interpolate the crop coefficients

doy_start = int(DATE_START.strftime('%j'))
doy_k0 = int(DATE_SOWING.strftime('%j')) # Planting date
doy_k1 = int(datetime.date(2024, 4, 1).strftime('%j')) # Mature date
doy_k2 = int(datetime.date(2024, 6, 1).strftime('%j')) # Harvest date
doy_end = int(DATE_END.strftime('%j')) # To have a complete year
crop_coef_known = np.array([0.3, 1.15, 0.25])

# NOTE: Have to add a +1 to end because 2024 is a leap year
doy = np.concatenate([np.arange(doy_start, 366), np.arange(1, doy_end + 1)])
crop_coef = linear_interp(doy, crop_coef_known, [doy_k0, doy_k1, doy_k2])

# Get DOY of the planting date
doy.tolist().index(doy_k0)

# TODO Making sure that crop coef is zero prior to planting date
crop_coef[0:14] = 0

#### &#x1F3AF; Best Practice: Numeric data types

We should always pay attention to the numeric data type used to store values. While a higher number of bits encodes more precision for a number, it also takes up more space in memory. Our crop coefficients could probably be safely represented using just 16 bits, the minimum number for a decimal or "floating point" number. Compare the two examples below

In [ ]:
# Note memory-saving potential of changing the dtype

print('64-bit precision:', crop_coef[20:30])
crop_coef = crop_coef.astype(np.float16)
print('16-bit precitions:', crop_coef[20:30])

### Computing the root fraction

Next, we need to similarly interpolate the root fraction time series. We know what the root fraction is on specific days of the year, so we need to lienarly interpolate the values between those dates. The final plot, below, shows what this looks like alongside our crop coefficient.

In [ ]:
root_fraction = linear_interp(doy, [0.1, 1.0], [doy_k0, doy_k1])
root_fraction = root_fraction.astype(np.float16)
root_fraction[0:14] = 0.0

# Get human-readable date labels
date_labels = [
    datetime.datetime.strptime('2023-%03d' % d, '%Y-%j') if d >= doy_start\
    else datetime.datetime.strptime('2024-%03d' % d, '%Y-%j')
    for d in doy
]
pyplot.ylabel('Crop Coefficient or Root Fraction')
pyplot.plot(date_labels, crop_coef, 'k-', label = 'Crop Coefficient')
pyplot.plot(date_labels, root_fraction, 'r-', label = 'Root Fraction')
pyplot.legend()
pyplot.show()

While the crop coefficient represents the general ability of the crop to take up and transpire water from the soil, the root fraction represents the percentage of the root depth explored by the crop at its current stage. Both increase during the growing season up to maturity because an increase in the root fraction means more soil water can be exploited. At maturity, for a crop like Spring Wheat, the crop coefficient declines steeply because the standing crop is mostly investing nutrients in the grain.

---

## Our first implementation

We're finally ready to start writing up the WRSI algorithm. An essential first step is to design the **interface** for our algorithm; i.e., what will the WRSI function look like? 

We usually design the interface first because our algorithm is just one part of a larger workflow or software system. But another good reason to start with the interface is that it represents essential details that aren't dependent on the implementation of the algorithm. Therefore, no matter what choices we make when writing our algorithm code, we have sensible boundary conditions to guide our work. Specifically, any algorithm has:

- Required inputs
- Required outputs
- Options or parameters that change the behavior

Desiging an interface in Python is as simple as writing the function signature, using the `def` keyword, and adding a docstring. For example:

In [ ]:
def water_requirement_satisfaction_index(
        doy, crop_coef, pet, precip, root_fraction, sw_init, 
        sw_frac = 0.45, field_capacity = 0.153):
    '''
    Computes the water requirement satisfaction index (WRSI), based on Senay
    and Verdin (2003):

        Senay, G. B., and J. Verdin. 2003. Characterization of yield 
        reduction in Ethiopia using a GIS-based crop water balance model. 
        Canadian Journal of Remote Sensing 29 (6):687–692.

    The default values for the `field_capacity` and the soil-water level 
    as a fraction of field capacity, `sw_frac`, are taken from the FAO 56
    manual, Tables 19 and 22, respectively, for Spring Wheat:

        Allen, R. G., L. S. Pereira, D. Raes, and M. Smith. 1998. 
        Crop evapotranspiration - Guidelines for computing crop water requirements. 
        Rome, Italy: FAO - Food and Agriculture Organization of the United Nations.
        
    
    Parameters
    ----------
    doy : Sequence
        The day of year (DOY) for each time step
    crop_coef : Sequence
        A sequence of crop coefficients for each time step
    pet : Sequence
        The potential evapotranspiration (mm day-1) for each day of simulation
    precip : Sequence
        The daily precipitation (mm day-1) for each day of simulation
    root_fraction : Sequence
        The root depth fraction, a dimensionless number between 0.0 and 1.0,
        for each day of simulation
    sw_init : Number
        The initial soil water level (mm), on the first day of simulation
    sw_frac : Number
        The soil water level, as a fraction of field capacity, below which
        AET becomes less than PET during the crop's mature stage; should be
        1.0 minus the allowable depletion fraction (dim.)
    field_capacity : Number
        The field capacity (mm)
    
    Returns
    -------
    Sequence
        The WRSI on each day of the simulation (mm day-1)
    '''
    pass

&#x1F449; **Note that:**

- We included citations for key background information, an essential justification for the choices that we made.
- The physical units for all quantities are described.

The `pass` statement isn't strictly necessary but it's a good idea. We now have a function that doesn't do anything but which is very specific about what the required inputs are, what the outputs should be (as described in the docstring), and what optional parameters (and their default values) are available.

### The WRSI algorithm

Below is one implementation of the WRSI algorithm.

&#x1F449; **Note that:**

- We included several comments, thoughout, to help a reader understand the code.
- Comments sometimes include a reference to the **authoritative description** (in this case, the Senay & Verdin 2003 paper), often indicating which specific equation(s) that section of the code implements. This makes it much easier to verify that the code is correct.
- Multiple `assert` statements at the top of the code helps to formalize expectations about the input datasets. If the user provides an input dataset in the incorrect format, they get an informative error message to help them fix the problem.

In [ ]:
def water_requirement_satisfaction_index(
        doy, crop_coef, pet, precip, root_fraction, sw_init, 
        sw_frac = 0.45, field_capacity = 0.153):
    '''
    Computes the water requirement satisfaction index (WRSI), based on Senay
    and Verdin (2003):

        Senay, G. B., and J. Verdin. 2003. Characterization of yield 
        reduction in Ethiopia using a GIS-based crop water balance model. 
        Canadian Journal of Remote Sensing 29 (6):687–692.

    The default values for the `field_capacity` and the soil-water level 
    as a fraction of field capacity, `sw_frac`, are taken from the FAO 56
    manual, Tables 19 and 22, respectively, for Spring Wheat:

        Allen, R. G., L. S. Pereira, D. Raes, and M. Smith. 1998. 
        Crop evapotranspiration - Guidelines for computing crop water requirements. 
        Rome, Italy: FAO - Food and Agriculture Organization of the United Nations.

        
    Parameters
    ----------
    doy : Sequence
        The day of year (DOY) for each time step
    crop_coef : Sequence
        A sequence of crop coefficients for each time step
    pet : Sequence
        The potential evapotranspiration (mm day-1) for each day of simulation
    precip : Sequence
        The daily precipitation (mm day-1) for each day of simulation
    root_fraction : Sequence
        The root depth fraction, a dimensionless number between 0.0 and 1.0,
        for each day of simulation
    sw_init : Number
        The initial soil water level (mm), on the first day of simulation
    sw_frac : Number
        The soil water level, as a fraction of field capacity, below which
        AET becomes less than PET during the crop's mature stage; should be
        1.0 minus the allowable depletion fraction (dim.)
    field_capacity : Number
        The field capacity (mm)
    
    Returns
    -------
    Sequence
        The WRSI on each day of the simulation (mm day-1)
    '''
    assert crop_coef.ndim == 1, 'crop_coef should be a 1-dimensional numeric sequence'
    assert pet.ndim == 1, 'pet should be a 1-dimensional numeric sequence'
    assert precip.ndim == 1, 'precip should be a 1-dimensional numeric sequence'
    assert root_fraction.ndim == 1, 'root_fraction should be a 1-dimensional numeric sequence'
    # Pre-allocate vectors for holding data for each time step
    paw = np.nan * np.ones(pet.shape, dtype = np.float32) # Plant available water
    sw  = np.nan * np.ones(pet.shape, dtype = np.float32) # Soil water
    aet_c = np.nan * np.ones(pet.shape, dtype = np.float32) # AETc
    # Compute PETc at each time step (Eq. 2, Senay & Verdin 2003)
    pet_c = pet * crop_coef # PETc
    # Compute the critical soil water at each time step
    #    (Eq. 7, Senay & Verdin 2003)
    sw_crit = field_capacity * sw_frac * root_fraction
    for t in range(len(pet)):
        # Compute plant available water (PAW) (Eq. 3, Senay & Verdin 2003)
        if t == 0:
            paw[t] = sw_init + precip[t] # At t=0, sw[t-1] is unknown
        else:
            paw[t] = sw[t-1] + precip[t]
        # Compute AET (Eq. 4-6, Senay & Verdin 2003); above the critical
        #    soil water level, this is just PET
        if paw[t] >= sw_crit[t]:
            aet_c[t] = pet_c[t]
        else:
            aet_c[t] = (paw[t] / sw_crit[t]) * pet_c[t]
        aet_c[t] = np.min([aet_c[t], paw[t]]) # Cannot be higher than PAW
        # Compute (remaining) soil water
        sw[t] = paw[t] - aet_c[t]
        sw[t] = np.min([sw[t], field_capacity]) # Cannot be higher than FC
        sw[t] = np.max([sw[t], 0.0])
    
    # At the end, compute WRSI on each day of the simulation; i.e.,
    #    compute totals on each day (a cumulative sum)
    wrsi = (np.cumsum(aet_c) / np.cumsum(pet_c)) * 100
    return wrsi.astype(np.float32)

How does this funciton work? Let's try it out with some synthetic data. This could form the basis for a **unit test** that we can later use to verify that our function is still working as intended.

In [ ]:
# Random precipitation time series, between 0 and 3 mm
precip = np.random.choice([0, 1, 2, 3], size = doy.size, p = [0.6, 0.3, 0.08, 0.02])
pet = 3 * np.ones(doy.size) # Constant 3 mm of PET

wrsi = water_requirement_satisfaction_index(
    doy[14:], crop_coef[14:], pet[14:], precip[14:], root_fraction[14:], 
    sw_init = 10, sw_frac = 0.45, field_capacity = 0.153)

In [ ]:
pyplot.plot(date_labels[15:], wrsi[1:], 'k-')

---

## Applying our function across space

Iterative algorithms like the WRSI don't work well with `xarray` because they consist of more than series of algebraic operations. Consequently, in order to apply our algorithm to the millions of pixels in our study's spatial domain, we'll need to find a way to scale-up our analysis without using `xarray` and its built-in compatibility with `dask`.

One way to address this issue is treat the pixels of our spatial domain as just elements of a big series, where each pixel can be processed independently. We'll need to reshape our input datasets from 3-dimensional, spatially explicit maps to a collection of 1-dimensional time series.

For example, our PET data are currently in the form `(T, Y, X)`, a 3-dimensional array.

In [ ]:
pet = ds_et_daily.sel(time = slice(DATE_START, DATE_END)).PET.values
pet.shape

We want to reshape this to a 2-dimensional `(T x N)` array, for `T` time steps and `N` pixels.

In [ ]:
# Reshaping to a (T x N) array for N pixels
days, rows, cols = pet.shape
pet_raveled = pet.reshape((days, rows * cols))
pet_raveled.shape

As we can see, there are about 1.6 million pixels on our spatial domain. But wait---many of these pixels can be ignored because they're water pixels or correspond to a land-cover other than croplands. Can we use our land-cover map to filter out these pixels?

In [ ]:
# Note that lc_map has same shape as our raveled spatial domain

lc_map.ravel().shape

In [ ]:
# Filtering the NoData pixels out using the land-cover map
pet_series = pet_raveled[:,lc_map.ravel() == 1]
pet_series.shape

Since `lc_map` has the same number of elements (pixels) as our PET data, we use it to filter the PET data. 65,000 pixels is better than 1.6 million!

Below, we repeat this for the precipitation data and our initial soil water level dataset.

In [ ]:
precip_series = ds_precip_1km.sel(time = slice(DATE_START, DATE_END)).precipitation.values\
    .reshape((days, rows * cols))
precip_series = precip_series[:,lc_map.ravel() == 1]
print(precip_series.shape)

swc_series = swc.where(swc >= 0, 0).values.reshape((rows * cols,))
swc_series = swc_series[lc_map.ravel() == 1]
print(swc_series.shape)

**We're now ready to compute the WRSI for all cropland pixels.**

This next part could take about 6-8 minutes.

In [ ]:
from tqdm import tqdm

# Get index of first time step when crop is sown
t = doy.tolist().index(int(DATE_SOWING.strftime('%j')))

# Get a data structure to hold WRSI outputs
wrsi = np.nan * np.ones(precip_series[t:,].shape, dtype = np.float32)

for i in tqdm(range(precip_series.shape[1])):
    wrsi[:,i] = water_requirement_satisfaction_index(
        doy[t:], crop_coef[t:], pet_series[t:,i], precip_series[t:,i], 
        root_fraction[t:], sw_init = swc_series[i], sw_frac = 0.45, field_capacity = 0.153)

### Converting 1-dimensional output back to 2-dimensional space

**We filtered out pixels to just cropland pixels so that we'd have smaller, 1-dimensional time series data to work with.** How do we recover the 2-dimensional spatial representation we lost?

The answer is that we use `lc_map` again, reversing the operation. Here, we create an empty 2-dimensional map (really, a `(T, Y, X)` array) and place the `T`-length time series into the map, using `lc_map` to indicate where the data go. Because `numpy` always does indexing the same way, we can use the same filter expression, `lc_map == 1`, to refer to the same pixel locations.

In [ ]:
wrsi_map = np.nan * np.ones((wrsi.shape[0], *lc_map.shape), dtype = np.float32)
wrsi_map[:,lc_map == 1] = wrsi
wrsi_map.shape

In [ ]:
pyplot.imshow(wrsi_map[-1], interpolation = 'nearest', vmin = 0)
pyplot.colorbar()

We can convert our WRSI data into an `xarray` dataset to make it easier to subset and plot the data.

In [ ]:
ds_wrsi = xr.Dataset(
    data_vars = {'WRSI': (['time', 'y', 'x'], wrsi_map)},
    coords = {
        'time': ds_et_daily.time[(t+1):],
        'x': ds_et_daily.x,
        'y': ds_et_daily.y
    })

Let's zoom-in on the Tunis region of northern Algeria and look at a map of the WRSI on the date we estimated that spring wheat reaches maturity, April 1, 2024. We can see a broad north-south gradient of WRSI, where croplands in the north are closer to meeting their water requirements that those in the south. This is consistent with a broad north-south gradient in precipitation and PET.

In [ ]:
ds_wrsi.sel(y = slice(4.45e6, 4.225e6), x = slice(0.8e6, 1e6), time = '2024-04-01').WRSI.plot(vmin = 0)

Now that we have an `xarray` dataset, we can also pull out a time series for a specific location.

In [ ]:
tunis_coords = {'x': 927211, 'y': 4414719}

ds_wrsi.sel(**tunis_coords, method = 'nearest').WRSI.plot()

### Comparing Tunis and Tiaret

We can compare the WRSI in two different locations, for example, at Tunis and Tiaret. We can see from the plot below that, while both locations experience similar variability in precipitation and PET, Tunis is generally wetter than Tiaret and so it has a higher WRSI throughout the growing season.

In [ ]:
tiaret_coords = {'x': 141143, 'y': 4244081}

tunis_wrsi = ds_wrsi.WRSI.sel(**tunis_coords, method = 'nearest').values
tiaret_wrsi = ds_wrsi.WRSI.sel(**tiaret_coords, method = 'nearest').values

pyplot.plot(date_labels[t:], tunis_wrsi, label = 'Tunis')
pyplot.plot(date_labels[t:], tiaret_wrsi, label = 'Tiaret')
pyplot.legend()
pyplot.show()

---

## References

- Fan, J., McConkey, B., Wang, H., & Janzen, H. (2016). Root distribution by depth for temperate agricultural crops. *Field Crops Research,* 189, 68-74. https://doi.org/10.1016/j.fcr.2016.02.013
- FAO 56, Table 19, "Typical soil water characteristics for different soil types," https://www.fao.org/4/X0490E/x0490e0c.htm#soil%20evaporation%20reduction%20coefficient%20(kr)
- FAO 56, Table 22, "Ranges of maximum effective rooting depth...for common crops," https://www.fao.org/4/X0490E/x0490e0e.htm#readily%20available%20water%20(raw)